# Assignment 03

### Deadline: March 6th 2025, 10:00 AM

### Deliverables:
Please submit, via Canvas, a **zip file** including the following:
- a .ipynb file (you can use this one) with all of your code included -- `03_assignment_surname1_surname2.ipynb`
- a compiled .html file of your .ipynb which includes all of the output -- `03_assignment_surname1_surname2.html`
- a pdf file with your written answers to the questions -- `03_assignment_surname1_surname2.pdf`

Make sure to follow the naming convention indicated above. The zip name can be named `03_assignment_surname1_surname2`.

Make sure to annotate your code. We may substract points if code is not annotated and unclear.

To be able to run this notebook, run the following commands in your virtual environment: 

```
# navigate to your folder with the virtual environment OR create a new one
cd <.../.../.../04_otherXAI>

# activate the virtual environment 
source <venv_name>/bin/activate

# or on Windows: 
<venv_nane>/Scripts/activate

# install a package to for visualisation. My version of mlxtend is 0.23.4, of gplearn it's 0.4.2, and of pysr it's 0.19.4
pip install mlxtend gplearn pysr

# install the package for robust counterfactuals
git clone https://github.com/donato-maragno/robust-CE.git

cd robust-CE

pip install .

# close the virtual environment
deactivate

```

## Data

We are using the Statlog (German Credit Data) dataset. The German Credit dataset classifies people described by a set of 20 features as good or bad credit risk.

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import math

In [8]:
# train_data 
X_train = pd.read_csv('../datasets/credit/encoded_credit_X_train.csv')
y_train = pd.read_csv('../datasets/credit/credit_y_train.csv')

# test data
X_test = pd.read_csv('../datasets/credit/encoded_credit_X_test.csv')
y_test = pd.read_csv('../datasets/credit/credit_y_test.csv')

# Robust Counterfactuals

## 1. Preparation

(5 points)

For demonstration purposes we will train a model to predict good/bad credit risk using only `duration` and `credit_amount`, for which we will then create robust counterfactual explanations.

**1.1** Select only the features `duration` and `credit_amount` from the dataset. Then, apply `MinMaxScaler` from `sklearn` to scale the data to be between 0 and 1.

**1.2** Train a decision tree using `DecisionTreeClassifier` from `scikit-learn`. Use `max_depth = 10` and `random_state = 1`.

**1.3** Inspect the model and evaluate performance and interpretability.

In [ ]:
## ****************************************************************************
##                           adjust code here for Q 1.1, Q 1.2 and Q 1.3
## ****************************************************************************

# -------------
# select only duration and credit_amount
# -------------
...

# -------------
# scale the data
# -------------
from sklearn.preprocessing import MinMaxScaler

...


## ****************************************************************************
##                           adjust code here for Q 1.1 and Q 1.2
## ****************************************************************************

# -------------
# train model
# -------------

# import decision tree class from sklearn
...

# initialize and fit classifier
...

# predict on test set 
...


# -------------
# inspect performance
# -------------

# import and print metrics
...

## 2. Optimization

(10 points)

We are using sample 54 as example datapoint to explain. The prediction for this sample is 0, so we want to know how to change the features such that the prediction would be 1.

**2.1** Run `rce.generate()` to generate a region of counterfactual explanations, i.e. robust counterfactuals for the sample at index 54. Follow the readme file on the repo: https://github.com/donato-maragno/robust-CE/. Use the iterative approach. As uncertainty set, we are using the L-infinity norm. Choose the radius of the set to be 0.05. 

**2.2** The solution returns the center of the set. Given this and the radius of the uncertainty set, calculate the lower and upper bound of the box. We want to save those in a dataframe, balled `bounds`, where the columns are `duration` and `credit_amount`. The first row should show the lower bound of each feature respectively, and the second row should show the upper bound, like in this example:

<div>
<img src="../img/bounds example.png" width="250" align='left'/>
</div>

<br/><br/>
<br/><br/>

Once you have done that you can apply reverse scaling to this dataframe to inspect the feature ranges in the original feature space.

**2.3** Plot the decision areas of the decision tree, the factual instance and the box of the counterfactuals. You can use the code provided below. (Note: use the scaled data)

In [ ]:
# -------------
# choose factual instance
# -------------
idx = 54
print('The actual label of sample {0} is {1}.'.format(idx, y_test.iloc[idx,-1]))
print('The predicted label of sample {0} is {1}.'.format(idx, y_pred.loc[idx]))
# define the factual instance
u = pd.DataFrame(np.array(X_test.loc[idx,:]).reshape(1,-1), columns=X_test.columns)
u

In [ ]:
## ****************************************************************************
##                           adjust code here for Q 2.1
## ****************************************************************************

# import robust CE package
import rce

# assign all parameters
...

# run model
...


## ****************************************************************************
##                           adjust code here for Q 2.2
## ****************************************************************************

# create dataframe with lower and upper bounds of the box

...

bounds = ...

In [ ]:
## ****************************************************************************
##                           code for Q 2.1 (ideally no need to adjust)
## ****************************************************************************

# -------------
# visualise results
# -------------

import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from matplotlib.patches import Rectangle

# features to plot
features_to_plot = {'x':'duration','y':'credit_amount'}

# details for plotting datapoints
scatter_kwargs = {'s': 80, 'edgecolor': None, 'alpha': 0.6}
contourf_kwargs = {'alpha': 0.2}
scatter_highlight_kwargs = {'s': 120, 'label': 'factual instance', 'alpha': 1.0, 'c':'yellow', 'edgecolor':'darkgrey'}

# Create figure and axes
fig, ax = plt.subplots(1,figsize=(10, 10))

# plot decision areas
plot_decision_regions(np.array(X_test), np.array(y_test).squeeze(), 
                      clf=clf, 
                      legend=2, # top left 
                      X_highlight=np.array(u), # factual instance
                      zoom_factor=1, # level of detail
                      scatter_kwargs=scatter_kwargs, # size, color, etc. of datapoints
                      contourf_kwargs=contourf_kwargs,
                      scatter_highlight_kwargs=scatter_highlight_kwargs) # size, color, etc. of factual instance

# adding robust CE box
ax.add_patch(Rectangle((bounds.loc['lower_bound'][features_to_plot['x']], 
                        bounds.loc['lower_bound'][features_to_plot['y']]), 
                       rho*2, rho*2,
                       color='green',
                       edgecolor = 'black',
                       fill=True,
                       lw=0.5,
                       alpha=0.6))

# Adding axes annotations
plt.xlabel(f'scaled {features_to_plot["x"]}')
plt.ylabel(f'scaled {features_to_plot["y"]}')
plt.title('Decision areas of the decision tree')
plt.xlim(-0.05, 0.8)
plt.ylim(-0.05, 0.8)

plt.show()

## 3. Discussion 

(10 points)

**3.1** Describe the results – what does the function return, what does the resulting box tell us, how can we formulate an explanation from it? Use the results for sample at index 54 to support your answer.

**3.2** In some situations the iterative approach may take quite long. In our paper we describe a heuristic variant that solves the problem much quicker. Describe this heuristic and explain why it is quicker, and what trade-off we might run into. You can find the paper here: https://pubsonline.informs.org/doi/abs/10.1287/ijoc.2023.0153?journalCode=ijoc. It is also uploaded to the Github repo.

**3.3** Run the code again, now using the heuristic. Explain the result and compare the result with the previous one.

---
# Symbolic regression

In this assignment you will again compare GPLearn with PySR by choosing an equation from the AI Feynman Physics-Inspired Database of Equations (Udrescu & Tegmark, 2019). You will download the dataset and load it into this assignment notebook. 

Open the AI Feynman database of equations. You can find it here: https://space.mit.edu/home/tegmark/aifeynman.html. In Table IV of the provided paper, or in folder `'Feynman_with_units.tar.gz'`, look for a monomial with at least four features. You can download the dataset that belongs to the chosen equation from `'Feynman_with_units.tar.gz'`. 

**IMPORTANT:** The dataset is quite big and you may have problems downloading it. Hence, we have uploaded a subset of the equations to Canvas (under week 4). You can select an **equation with 4 features** from this subset. Make sure to download the folder and place it in the datasets folder of the repo :) 

## 4. Feynman Symbolic Regression Database

(15 points)

**4.1** Create a dataset of your chosen equation as a DataFrame. Take only the first 30 samples of the dataset of your chosen equation. This requires you to download the dataset from the AI Feynman webpage first. Remember to choose an equation with at least four features.

**4.2** Use GPLearn to find the equation you chose. Motivate the chosen values for `generations` and `population_size`.

**4.3** Another package that fits a symbolic regression model is `PySR`. Install it using `pip install pysr` if you haven't done that yet. Fit their `PySRRegressor` to the dataset. Follow the quick start instructions on their Github repo: https://github.com/MilesCranmer/PySR. The complete documentation can be found here: https://astroautomata.com/PySR/

- for the arguments `binary_operators` and `unary_operators` you can use the default values
- make sure that your model is reproducible. Read the documentation on how to do that for this model. 

**4.4** Create a synthetic test dataset based on the true equation consisting of 10 samples and use both found equations to predict the corresponding target value for each sample. Provide the total L1-loss for both GPLearn PySR. Recall that you can calculate the total L1-loss by simply summing up the differences between the actual output and predicted output for each sample. What are your conclusions about both methods?

In [ ]:
## ****************************************************************************
##                        adjust/add code here for Q 4.1
## ****************************************************************************

# load the dataset for the equation that you chose
# you can use the code below

# choose equation
equation_label = '...'

file_path = f'../datasets/feynman_equations/{equation_label}'
data = pd.read_csv(file_path, header=None, sep = ' ').iloc[:,:-1] # remove last column because feynman datasets include empty space in the last column
data.columns = ['x1','x2','x3','x4','output'] # add column names

# select only first 30 rows
...

# separate input features and output
X = ...
y = ...


## ****************************************************************************
##                        adjust/add code here for Q 4.3
## ****************************************************************************

from gplearn.genetic import SymbolicRegressor

# fit SymbolicRegressor
...

# get equation
...


## ****************************************************************************
##                        adjust/add code here for Q 4.4
## ****************************************************************************

from pysr import PySRRegressor

# fit PySRRegressor
...

# get best equation
...


## ****************************************************************************
##                        adjust/add code here for Q 4.5
## ****************************************************************************

# Creating dataset according to the equation

random_state = 42
nof_samples = 10
dim = 4
np.random.seed(random_state)

# generate data
...

# calculate y_actual using the true equation & predict y using both equations returned by the models
...

# calculate loss for both models
...